# Parsing Puzzle

In [92]:
def parse_pips_puzzle(filename):
    with open(filename, 'r') as f:
        # read rows/cols
        n = int(f.readline().strip())

        # read board matrix
        board = []
        for _ in range(n):
            row = list(f.readline().strip())
            line = []
            for c in row:
                if c == '…':
                    line.extend(['.', '.', '.'])
                else:
                    line.append(c)
            
            board.append(line)

        # read number of constraints
        c = int(f.readline().strip())

        constraints = {}
        for _ in range(c):
            parts = f.readline().strip().split()
            letter = parts[0]
            ctype = parts[1]     # EQ, NEQ, LT, GT, SUM
            if ctype == "SUM":
                value = int(parts[2])
            elif ctype in ("EQ", "NEQ", "LT", "GT"):
                value = int(parts[2])
            else:
                raise ValueError("Unknown constraint type: " + ctype)

            constraints[letter] = (ctype, value)

        # read number of dominos
        m = int(f.readline().strip())

        dominos = []
        for _ in range(m):
            a, b = f.readline().strip().split()
            dominos.append((int(a), int(b)))

        return n, board, constraints, dominos

Below, can input file containing the puzzle and it will be parsed and stored

In [93]:
n, board, constraints, dominos = parse_pips_puzzle('pips1.txt')
print("Board size:", n)
print("Board:")
for row in board:
    print(' '.join(row))
print("Constraints:", constraints)
print("Dominos:", dominos)

Board size: 4
Board:
A # # B
. . . B
. . C C
. . C .
Constraints: {'A': ('SUM', 3), 'B': ('SUM', 11), 'C': ('SUM', 15)}
Dominos: [(5, 1), (6, 5), (3, 0), (5, 5)]


# Writing Variables and Constraints

## Variables

We model the Pips puzzle as a constraint satisfaction problem using two types of decision variables.
First, each domino is represented by a single integer variable place[d] whose domain consists of all legal placements of that domino on the board; each placement encodes a row, a column, and an orientation, and immediately determines which two cells the domino covers and which pip values it contributes.
Second, for each valid board cell we introduce an integer variable cell_value[r][c] representing the pip value assigned to that cell.
The link between these two variable types will be enforced by conditional constraints: whenever a domino chooses a particular placement, it sets the pip values of the two cells it covers accordingly.

In [94]:
from ortools.sat.python import cp_model

def define_variables(model, n, board, dominos):
    m = len(dominos)
    maxpip = max(max(d) for d in dominos)  # highest pip value

    # 1. Generate all valid placements
    placements = [[] for _ in range(m)]

    for d, (v0, v1) in enumerate(dominos):
        # Horizontal placements
        for r in range(n):
            for c in range(n - 1):
                if board[r][c] != '.' and board[r][c+1] != '.':
                    placements[d].append((r, c, 0, v0, v1))  # forward
                    placements[d].append((r, c, 1, v1, v0))  # reverse

        # Vertical placements
        for r in range(n - 1):
            for c in range(n):
                if board[r][c] != '.' and board[r+1][c] != '.':
                    placements[d].append((r, c, 2, v0, v1))  # forward
                    placements[d].append((r, c, 3, v1, v0))  # reverse

    # 2. Create domino placement variables
    place = []
    for d in range(m):
        K = len(placements[d])
        var = model.NewIntVar(0, K - 1, f"place_{d}")
        place.append(var)

    # 3. Create cell pip-value variables
    cell_value = [[None]*n for _ in range(n)]

    for r in range(n):
        for c in range(n):
            if board[r][c] == '.':
                continue   # unusable cell -> no variable
            cell_value[r][c] = model.NewIntVar(0, maxpip, f"cell_{r}_{c}")

    return place, placements, cell_value, maxpip


## Constraints

For our constraints, we need to ensure that:
1. Every valid cell has exactly one variable.
2. Every domino has been placed.
3. The solution is valid and dominos have been placed correctly (i.e. can't split up the halves of the domino)
4. The constraints given in the Pips problem have been met.

1. Ensuring that every valid cell has exactly one variable:
- For each domino placement we create a Boolean indicator variable.
- For each cell, we gather all domino placements that physically cover that cell.
- Using AddExactlyOne, we enforce that exactly one of those placements is chosen.
- This ensures every valid cell is covered once and only once by dominos.

In [95]:
def add_cell_coverage_constraints(model, n, board, place, placements, cell_value):
    m = len(placements)

    # Create Boolean indicator variables for each placement
    is_place = []
    for d in range(m):
        K = len(placements[d])
        is_place_d = []
        for k in range(K):
            b = model.NewBoolVar(f"is_place_{d}_{k}")
            model.Add(place[d] == k).OnlyEnforceIf(b)
            model.Add(place[d] != k).OnlyEnforceIf(b.Not())
            is_place_d.append(b)
        is_place.append(is_place_d)

    # For every valid cell, enforce exactly 1 covering domino
    for r in range(n):
        for c in range(n):
            if board[r][c] == '.':
                continue  # unusable cell, skip

            covering_literals = []

            # Find all domino placements that cover this cell
            for d in range(m):
                for k, (rr, cc, o, v0, v1) in enumerate(placements[d]):
                    # Domino placement covers two cells depending on orientation
                    if o == 0:  # horizontal forward
                        cells = [(rr, cc), (rr, cc+1)]
                    elif o == 1:  # horizontal reverse
                        cells = [(rr, cc), (rr, cc+1)]
                    elif o == 2:  # vertical forward
                        cells = [(rr, cc), (rr+1, cc)]
                    else:  # o == 3, vertical reverse
                        cells = [(rr, cc), (rr+1, cc)]

                    # If this placement covers (r,c), include it
                    if (r, c) in cells:
                        covering_literals.append(is_place[d][k])

            # Exactly one domino covers this cell
            model.AddExactlyOne(covering_literals)

    return is_place


2. Ensuring that every domino is placed exactly once.
In the CP-SAT model, each domino is represented by a single integer variable place[d] whose domain is the set of all legal placements for that domino. Because this variable must take exactly one value in any solution, we no longer need a separate “exactly once” constraint for each domino.

3. The solution is valid and dominos have been placed correctly (i.e. can't split up the halves of the domino).
We can encode this with implication. I.e. if a domino is placed, the exact two cells must contain its two halves.

We already have a constraint saying that each domino can only be placed once, and these two in coordination ensure that our solution is feasible.
Although coverage constraints ensure that each cell is occupied exactly once, they do not specify which pip value appears in each cell. To guarantee that each domino contributes the correct two pip values according to the chosen placement, we add a channeling constraint: whenever place[d] selects a placement k, the pip values of the two cells covered by that placement are fixed to the domino’s values. This prevents dominos from being “split,” rotated inconsistently, or assigned incorrect values.

In [96]:
def add_domino_value_constraints(model, n, placements, place, is_place, cell_value):
    m = len(placements)

    for d in range(m):                 # each domino
        for k, (r, c, o, v0, v1) in enumerate(placements[d]):

            # If placement k is chosen, enforce value on first cell
            model.Add(cell_value[r][c] == v0).OnlyEnforceIf(is_place[d][k])

            # Determine the second cell based on orientation
            if o == 0 or o == 1:       # horizontal (forward or reverse)
                rr, cc = r, c+1
            else:                      # o == 2 or 3 (vertical)
                rr, cc = r+1, c

            # If placement k is chosen, enforce value on second cell
            model.Add(cell_value[rr][cc] == v1).OnlyEnforceIf(is_place[d][k])



4. The constraints given in the Pips problem have been met.


In [97]:
def add_region_EQ(model, cells, cell_value):
    if len(cells) <= 1:
        return
    (r0, c0) = cells[0]
    base = cell_value[r0][c0]
    for (r, c) in cells[1:]:
        model.Add(cell_value[r][c] == base)


In [98]:
def add_region_NEQ(model, cells, cell_value):
    vars_ = [cell_value[r][c] for (r, c) in cells]
    model.AddAllDifferent(vars_)


In [99]:
def add_region_SUM(model, cells, target, cell_value):
    model.Add(sum(cell_value[r][c] for (r, c) in cells) == target)


In [100]:
def add_region_LE(model, cells, target, cell_value):
    model.Add(sum(cell_value[r][c] for (r, c) in cells) <= target)


In [101]:
def add_region_GE(model, cells, target, cell_value):
    model.Add(sum(cell_value[r][c] for (r, c) in cells) >= target)


In [102]:
def add_region_constraints(model, board, constraints, cell_value):
    n = len(board)

    # Build region → list of cells mapping
    regions = {}
    for r in range(n):
        for c in range(n):
            ch = board[r][c]
            if ch not in ('.', '#'):
                regions.setdefault(ch, []).append((r,c))

    # Add constraints
    for label, (ctype, value) in constraints.items():
        if label not in regions:
            continue
        cells = regions[label]

        if ctype == "EQ":
            add_region_EQ(model, cells, cell_value)

        elif ctype == "NEQ":
            add_region_NEQ(model, cells, cell_value)

        elif ctype == "SUM":
            add_region_SUM(model, cells, value, cell_value)

        elif ctype == "LE":
            add_region_LE(model, cells, value, cell_value)

        elif ctype == "GE":
            add_region_GE(model, cells, value, cell_value)

        else:
            raise ValueError(f"Unknown constraint type: {ctype}")

# Initializing Model and Solving the Puzzle

The solve_pips function will take in the board size, the board, the constraints, and the dominos (which our file parser sets up earlier).
Then, it generates teh variables and constraints and solves the puzzle

In [103]:
def solve_pips(n, board, constraints, dominos):
    model = cp_model.CpModel()

    # Variables & placements
    place, placements, cell_value, maxpip = define_variables(model, n, board, dominos)

    # Constraints:
    # 1. Every valid cell is covered exactly once
    is_place = add_cell_coverage_constraints(model, n, board, place, placements, cell_value)

    # 2. Domino placement ⇒ pip values in the two cells
    add_domino_value_constraints(model, n, placements, place, is_place, cell_value)

    # 3. Region constraints
    add_region_constraints(model, board, constraints, cell_value)

    # Solve
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status not in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        print("No solution found.")
        return None

    # Build solution grid of pip values (or None)
    sol_grid = [[None] * n for _ in range(n)]
    for r in range(n):
        for c in range(n):
            if board[r][c] == '.':
                sol_grid[r][c] = None
            else:
                sol_grid[r][c] = solver.Value(cell_value[r][c])

    return sol_grid, solver


Solving the puzzle we parsed in earlier:

In [104]:
sol, solver = solve_pips(n, board, constraints, dominos)
if sol is not None:
    print("Solution pip grid:")
    for row in sol:
        print(" ".join("." if v is None else str(v) for v in row))
    print(solver.ResponseStats())

{'A': ('SUM', 3), 'B': ('SUM', 11), 'C': ('SUM', 15)}
Solution pip grid:
3 0 1 5
. . . 6
. . 5 5
. . 5 .
CpSolverResponse summary:
status: OPTIMAL
objective: 0
best_bound: 0
integers: 0
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime: 0.023427
usertime: 0.023427
deterministic_time: 9.76e-06
gap_integral: 0
solution_fingerprint: 0x27e5732c64fe3957

